<a href="https://colab.research.google.com/github/tonycico0912/ChatGPT_Python/blob/main/%EA%B5%AD%EB%82%B4%EC%A3%BC%EC%8B%9D%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%88%98%EC%A7%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install beautifulsoup4


In [5]:
import requests as rq  # requests 라이브러리를 rq라는 이름으로 가져옴
from bs4 import BeautifulSoup  # BeautifulSoup 라이브러리를 가져옴 : 이 라이브러리는 HTML을 파싱하고 정보를 추출하는데 사용됨

url = 'https://finance.naver.com/sise/sise_deposit.nhn'  # 스크래핑할 웹페이지의 URL을 정의
data = rq.get(url)  # requests의 get 함수를 사용하여 웹페이지의 HTML을 가져옴
data_html = BeautifulSoup(data.content)  # 가져온 HTML을 BeautifulSoup 객체로 변환하여 파싱할 수 있게 함

# 'div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah' CSS 선택자를 사용하여 웹페이지에서 특정 요소를 선택
# select_one 함수는 CSS 선택자에 일치하는 첫 번째 요소를 반환
parse_day = data_html.select_one('div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text

print(parse_day)  # 선택한 요소의 텍스트를 출력

import re  # re 모듈을 가져옴. re 모듈은 정규 표현식을 사용하여 문자열을 처리하는데 사용

# re.findall 함수를 사용하여 parse_day 문자열에서 숫자를 모두 찾음 '[0-9]+'는 하나 이상의 숫자를 의미하는 정규 표현식임
biz_day = re.findall('[0-9]+', parse_day)

# ''.join 함수를 사용하여 찾은 숫자들을 하나의 문자열로 합침
biz_day = ''.join(biz_day)

print(biz_day)  # 합친 문자열을 출력



  |  2023.07.11
20230711


In [6]:
import requests as rq  # 웹 요청을 보내기 위한 requests 라이브러리를 가져옴
from io import BytesIO  # 바이트 스트림을 처리하기 위한 io 라이브러리의 BytesIO를 가져옴
import pandas as pd  # 데이터 처리를 위한 pandas 라이브러리를 가져옴

###############################################################################
# KOSPI PART
###############################################################################

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'  # OTP를 생성하는 URL을 정의

# OTP 생성 요청에 필요한 파라미터를 정의
gen_otp_stk = {
    'mktId': 'STK',
    'trdDd': biz_day,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}

# 요청 헤더를 정의.  Referer 헤더는 요청이 어디서 시작되었는지를 나타냄
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}

# OTP 생성 URL에 POST 요청을 보냄.  요청의 본문에는 gen_otp_stk 딕셔너리를, 헤더에는 headers 딕셔너리를 사용
otp_stk = rq.post(gen_otp_url, gen_otp_stk, headers=headers).text

print(otp_stk)  # 생성된 OTP를 출력

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'  # 데이터를 다운로드하는 URL을 정의

# 다운로드 URL에 POST 요청을 보냄  요청의 본문에는 생성된 OTP를, 헤더에는 headers 딕셔너리를 사용
down_sector_stk = rq.post(down_url, {'code': otp_stk}, headers=headers)

# 다운로드한 데이터를 바이트 스트림으로 변환하고, 그 스트림을 pandas의 read_csv 함수에 전달하여 DataFrame으로 읽음
# 데이터의 인코딩은 'EUC-KR'임
sector_stk = pd.read_csv(BytesIO(down_sector_stk.content), encoding='EUC-KR')

sector_stk.head()  # DataFrame의 처음 5행을 출력.
sector_stk  # 전체 DataFrame을 출력



###############################################################################
# KOSDAQ PART
###############################################################################

# OTP 생성 요청에 필요한 파라미터를 정의. 이번에는 'mktId'를 'KSQ'로 설정하여 코스닥 데이터를 다운로드.
gen_otp_ksq = {
    'mktId': 'KSQ',  # 코스닥 입력
    'trdDd': biz_day,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}

# OTP 생성 URL에 POST 요청을 보냄. 요청의 본문에는 gen_otp_ksq 딕셔너리를, 헤더에는 headers 딕셔너리를 사용.
otp_ksq = rq.post(gen_otp_url, gen_otp_ksq, headers=headers).text

# 다운로드 URL에 POST 요청을 보냄. 요청의 본문에는 생성된 OTP를, 헤더에는 headers 딕셔너리를 사용
down_sector_ksq = rq.post(down_url, {'code': otp_ksq}, headers=headers)

# 다운로드한 데이터를 바이트 스트림으로 변환하고, 그 스트림을 pandas의 read_csv 함수에 전달하여 DataFrame으로 읽음
# 데이터의 인코딩은 'EUC-KR'
sector_ksq = pd.read_csv(BytesIO(down_sector_ksq.content), encoding='EUC-KR')

sector_ksq.head()  # DataFrame의 처음 5행을 출력

# pd.concat 함수를 사용하여 sector_stk DataFrame과 sector_ksq DataFrame을 연결.
# reset_index 함수를 사용하여 인덱스를 재설정함.  drop=True는 기존 인덱스를 삭제하라는 의미.
krx_sector = pd.concat([sector_stk, sector_ksq]).reset_index(drop=True)

# '종목명' 열의 값에서 앞뒤 공백을 제거함. str.strip 함수는 문자열의 앞뒤 공백을 제거하는 함수임
krx_sector['종목명'] = krx_sector['종목명'].str.strip()

# '기준일' 열에 biz_day 값을 설정함  이렇게 하면, 모든 행의 '기준일' 값이 biz_day 값으로 설정됨
krx_sector['기준일'] = biz_day

krx_sector.head()  # DataFrame의 처음 5행을 출력함
krx_sector


tuk3yvc955JHoOcLGBGa6CJPV8IaO6MZAqksqO+OqMARtSksuLS7Bnxpl86F7dAOkunw9BBwugQaSjGAcH15eRUzo0nT7RohfArSNnbp5m8tBgM+EFJCxYg3zco1gIgRZqIo4cIzoURnTI8+MmkJ4m8vFLhSKmM794gFu+ThsO31lY4woqehX8j6OlXFDcfHdV4NbYo4+D2Rwcfj24VnU3Zpq3ik/Dyw3FdyOXhJkBI=


,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,기준일
0,095570,AJ네트웍스,KOSPI,서비스업,4310,5,0.12,201804091450,20230711
1,006840,AK홀딩스,KOSPI,기타금융,18890,-110,-0.58,250246427290,20230711
2,027410,BGF,KOSPI,기타금융,3715,75,2.06,355587878565,20230711
3,282330,BGF리테일,KOSPI,유통업,168800,-1700,-1.00,2917523332800,20230711
4,138930,BNK금융지주,KOSPI,기타금융,6650,40,0.61,2167469385900,20230711
...,...,...,...,...,...,...,...,...,...
2601,024060,흥구석유,KOSDAQ,유통,5360,10,0.19,80400000000,20230711
2602,010240,흥국,KOSDAQ,기계·장비,6720,-60,-0.88,82808517120,20230711
2603,189980,흥국에프엔비,KOSDAQ,음식료·담배,2780,-60,-2.11,111583159060,20230711
2604,037440,희림,KOSDAQ,기타서비스,8300,-290,-3.38,115556542500,20230711


In [12]:
import requests as rq  # 웹 요청을 보내기 위한 requests 라이브러리를 가져옴
from io import BytesIO  # 바이트 스트림을 처리하기 위한 io 라이브러리의 BytesIO를 가져옴
import pandas as pd  # 데이터 처리를 위한 pandas 라이브러리를 가져옴

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'  # OTP를 생성하는 URL을 정의

# OTP 생성 요청에 필요한 파라미터를 정의
gen_otp_data = {
    'searchType': '1',
    'mktId': 'ALL',
    'trdDd': biz_day,
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03501'
}

# 요청 헤더를 정의. Referer 헤더는 요청이 어디서 시작되었는지를 나타냄.
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}

# OTP 생성 URL에 POST 요청을 보냄. 요청의 본문에는 gen_otp_data 딕셔너리를, 헤더에는 headers 딕셔너리를 사용
otp = rq.post(gen_otp_url, gen_otp_data, headers=headers).text

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'  # 데이터를 다운로드하는 URL을 정의

# 다운로드 URL에 POST 요청을 보냅 요청의 본문에는 생성된 OTP를, 헤더에는 headers 딕셔너리를 사용
krx_ind = rq.post(down_url, {'code': otp}, headers=headers)

# 다운로드한 데이터를 바이트 스트림으로 변환하고, 그 스트림을 pandas의 read_csv 함수에 전달하여 DataFrame으로 읽음
# 데이터의 인코딩은 'EUC-KR'입니다.
krx_ind = pd.read_csv(BytesIO(krx_ind.content), encoding='EUC-KR')

# '종목명' 열의 값에서 앞뒤 공백을 제거. str.strip 함수는 문자열의 앞뒤 공백을 제거하는 함수임
krx_ind['종목명'] = krx_ind['종목명'].str.strip()

# '기준일' 열에 biz_day 값을 설정. 이렇게 하면, 모든 행의 '기준일' 값이 biz_day 값으로 설정됨.
krx_ind['기준일'] = biz_day

krx_ind.head()  # DataFrame의 처음 5행을 출력
krx_ind

# set 함수를 사용하여 '종목명' 열의 값들을 집합으로 변환합니다. 집합은 중복된 값을 허용하지 않으므로, 각 집합에는 고유한 '종목명' 값들만 남게 됨
# symmetric_difference 메서드는 두 집합의 대칭 차집합을 반환합니다. 대칭 차집합은 두 집합에 한쪽에만 있는 요소들을 모은 집합임
# 즉, 이 코드는 한 DataFrame에는 있지만 다른 DataFrame에는 없는 '종목명' 값들을 찾습니다.
diff = list(set(krx_sector['종목명']).symmetric_difference(set(krx_ind['종목명'])))

print(diff)  # 찾은 '종목명' 값들을 출력

# pd.merge 함수를 사용하여 krx_sector DataFrame과 krx_ind DataFrame을 병합
# on 파라미터에는 병합의 기준이 되는 열의 이름들을 지정. 이 코드에서는 krx_sector와 krx_ind가 공통으로 가지고 있는 열의 이름들을 사용
# columns 속성은 DataFrame의 열의 이름들을 반환하고, intersection 메서드는 두 집합의 교집합을 반환. 따라서 이 코드는 krx_sector와 krx_ind가 공통으로 가지고 있는 열의 이름들을 찾음
# how 파라미터에는 병합의 방식을 지정. 'outer'는 외부 조인을 의미하므로, 한쪽에만 있는 행도 결과에 포함
kor_ticker = pd.merge(krx_sector,
                      krx_ind,
                      on=krx_sector.columns.intersection(
                          krx_ind.columns).tolist(),
                      how='outer')

# kor_ticker.head()  # 병합한 DataFrame의 처음 5행을 출력

print(kor_ticker[kor_ticker['종목명'].str.contains('건설|개발|신탁') & ~kor_ticker['종목명'].str.contains('기계')]['종목명'].values)

import numpy as np

kor_ticker['종목구분'] = np.where(kor_ticker['종목명'].str.contains('스팩|제[0-9]+호'), '스팩',
                              np.where(kor_ticker['종목코드'].str[-1:] != '0', '우선주',
                                       np.where(kor_ticker['종목명'].str.endswith('리츠'), '리츠',
                                                np.where(kor_ticker['종목명'].isin(diff),  '기타',
                                                '보통주'))))
kor_ticker = kor_ticker.reset_index(drop=True)
kor_ticker.columns = kor_ticker.columns.str.replace(' ', '')
kor_ticker = kor_ticker[['종목코드', '종목명', '시장구분', '종가',
                         '시가총액', '기준일', 'EPS', '선행EPS', 'BPS', '주당배당금', '종목구분']]
kor_ticker = kor_ticker.replace({np.nan: None})
kor_ticker['기준일'] = pd.to_datetime(kor_ticker['기준일'])

kor_ticker

from google.colab import files

file_name = input("Please enter the file name: ")
file_name_with_extension = f'{file_name}.xlsx'

kor_ticker.to_excel(file_name_with_extension, index=False)

files.download(file_name_with_extension)








['디앤디플랫폼리츠', '미래에셋맵스리츠', '네오이뮨텍', '로스웰', '엘브이엠씨홀딩스', 'NH프라임리츠', '코람코더원리츠', '엑세스바이오', 'GRT', '한국ANKOR유전', '코오롱티슈진', '맥쿼리인프라', 'ESR켄달스퀘어리츠', '윙입푸드', '골든센츄리', '크리스탈신소재', '애머릿지', 'JTC', 'KB스타리츠', '제이알글로벌리츠', '신한알파리츠', '미래에셋글로벌리츠', '맵스리얼티1', '코람코에너지리츠', '에이리츠', 'SBI핀테크솔루션즈', '마스턴프리미어리츠', '바다로19호', 'SK리츠', '이스트아시아홀딩스', '한화리츠', '에스씨엠생명과학', '이지스밸류리츠', '에스씨엠생명과학 [락]', '미투젠', '롯데리츠', '삼성FN리츠', '오가닉티코스메틱', '이지스레지던스리츠', '이리츠코크렙', '잉글우드랩', '케이탑리츠', 'NH올원리츠', '소마젠', '씨케이에이치', '헝셩그룹', '신한서부티엔디리츠', '한국패러랠', '글로벌에스엠', '컬러레이', '모두투어리츠', '프레스티지바이오파마']
['DL건설' 'GS건설' 'HDC현대산업개발' '계룡건설' '금호건설' '금호건설우' '대우건설' '동부건설' '동부건설우'
 '롯데관광개발' '삼호개발' '신세계건설' '일성건설' '태영건설' '태영건설우' '한국자산신탁' '한국토지신탁' '현대건설'
 '현대건설우' 'KCC건설' 'KH 건설' 'SGC이테크건설' '동신건설' '동원개발' '서희건설' '신원종합개발' '우원개발'
 '특수건설']
Please enter the file name: 테스트


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
import json
import requests as rq
import pandas as pd

url = f'''http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt={biz_day}&sec_cd=G10'''
data = rq.get(url).json()

type(data)
print(data.keys())

data['list'][0]

data['sector']

data_pd = pd.json_normalize(data['list'])

data_pd.head()

dict_keys(['info', 'list', 'sector', 'size'])


,IDX_CD,IDX_NM_KOR,ALL_MKT_VAL,CMP_CD,CMP_KOR,MKT_VAL,WGT,S_WGT,CAL_WGT,SEC_CD,SEC_NM_KOR,SEQ,TOP60,APT_SHR_CNT
0,G10,WICS 에너지,25449648,096770,SK이노베이션,8517928,33.47,33.47,1.0,G10,에너지,1,3,51780716
1,G10,WICS 에너지,25449648,009830,한화솔루션,4629496,18.19,51.66,1.0,G10,에너지,2,3,108292298
2,G10,WICS 에너지,25449648,010950,S-Oil,2690954,10.57,62.23,1.0,G10,에너지,3,3,41655633
3,G10,WICS 에너지,25449648,267250,HD현대,2623202,10.31,72.54,1.0,G10,에너지,4,3,44236128
4,G10,WICS 에너지,25449648,112610,씨에스윈드,2052229,8.06,80.61,1.0,G10,에너지,5,3,23615986


In [19]:
for i in range(1, 5):  # 1부터 4까지의 범위를 변경하여 필요한 범위를 확인합니다.
    url = f"http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt={biz_day}&sec_cd=G{i}"
    data = rq.get(url).json()
    print(f"For sec_cd=G{i}, data is: {data}")


For sec_cd=G1, data is: {'info': {'TRD_DT': '/Date(-62135596800000)/', 'MKT_VAL': 0, 'TRD_AMT': None, 'CNT': 0}, 'list': [], 'sector': [{'SEC_CD': 'G25', 'SEC_NM_KOR': '경기관련소비재', 'SEC_RATE': 0, 'IDX_RATE': 0}, {'SEC_CD': 'G35', 'SEC_NM_KOR': '건강관리', 'SEC_RATE': 0, 'IDX_RATE': 0}, {'SEC_CD': 'G50', 'SEC_NM_KOR': '커뮤니케이션서비스', 'SEC_RATE': 0, 'IDX_RATE': 0}, {'SEC_CD': 'G40', 'SEC_NM_KOR': '금융', 'SEC_RATE': 0, 'IDX_RATE': 0}, {'SEC_CD': 'G10', 'SEC_NM_KOR': '에너지', 'SEC_RATE': 0, 'IDX_RATE': 0}, {'SEC_CD': 'G20', 'SEC_NM_KOR': '산업재', 'SEC_RATE': 0, 'IDX_RATE': 0}, {'SEC_CD': 'G55', 'SEC_NM_KOR': '유틸리티', 'SEC_RATE': 0, 'IDX_RATE': 0}, {'SEC_CD': 'G30', 'SEC_NM_KOR': '필수소비재', 'SEC_RATE': 0, 'IDX_RATE': 0}, {'SEC_CD': 'G15', 'SEC_NM_KOR': '소재', 'SEC_RATE': 0, 'IDX_RATE': 0}, {'SEC_CD': 'G45', 'SEC_NM_KOR': 'IT', 'SEC_RATE': 0, 'IDX_RATE': 0}], 'size': [{'SEC_CD': 'WMI520', 'SEC_NM_KOR': 'WMI500 중형주', 'SEC_RATE': 0, 'IDX_RATE': 0}, {'SEC_CD': 'WMI530', 'SEC_NM_KOR': 'WMI500 소형주', 'SEC_RATE': 0,

In [20]:
import json
import requests as rq
import pandas as pd
from google.colab import files

# sec_cd 값과 해당하는 설명을 딕셔너리로 정의합니다.
sec_cd_dict = {
    "G25": "경기관련소비재",
    "G35": "건강관리",
    "G50": "커뮤니케이션서비스",
    "G40": "금융",
    "G10": "에너지",
    "G20": "산업재",
    "G55": "유틸리티",
    "G30": "필수소비재",
    "G15": "소재",
    "G45": "IT",
    "WMI520": "WMI500 중형주",
    "WMI530": "WMI500 소형주"
}

# 사용자에게 sec_cd 값을 선택하도록 안내합니다.
print("Please enter the sec_cd value from the following options:")
for key, value in sec_cd_dict.items():
    print(f"{key} ({value})")

# 사용자에게 sec_cd 값을 입력받습니다.
sec_cd = input()

# 사용자에게 파일명을 입력받습니다.
file_name = input("Please enter the file name: ")

# 입력받은 sec_cd 값을 URL에 포함시킵니다.
url = f'''http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt={biz_day}&sec_cd={sec_cd}'''

# URL에서 데이터를 가져옵니다.
data = rq.get(url).json()

# 데이터를 DataFrame으로 변환합니다.
data_pd = pd.json_normalize(data['list'])

# DataFrame을 엑셀 파일로 저장합니다.
data_pd.to_excel(f'{file_name}.xlsx', index=False)

# 엑셀 파일을 다운로드합니다.
files.download(f'{file_name}.xlsx')


Please enter the sec_cd value from the following options:
G25 (경기관련소비재)
G35 (건강관리)
G50 (커뮤니케이션서비스)
G40 (금융)
G10 (에너지)
G20 (산업재)
G55 (유틸리티)
G30 (필수소비재)
G15 (소재)
G45 (IT)
WMI520 (WMI500 중형주)
WMI530 (WMI500 소형주)
G40
Please enter the file name: 금융


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>